In [1]:
import datetime
import tensorflow as tf
from tensorflow import keras

# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO

- For this exercise, we will use the well-known MNIST digit data. 
- To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  
- Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 10

In [4]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3
input_shape = (img_rows, img_cols, 1)

In [5]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype("float32")
    x_test = x_test.astype("float32")
    x_train = x_train / 255
    x_test = x_test / 255
    print(f"x_train shape: {x_train.shape}")
    print(f"train samples: {x_train.shape[0]}")
    print(f"test samples: {x_test.shape[0]}")
    
    # convert class vectors to binary class matrices
    y_train = tf.keras.utils.to_categorical(train[1], num_classes)
    y_test = tf.keras.utils.to_categorical(test[1], num_classes)
    
    opt = tf.keras.optimizers.Adadelta()
    model.compile(loss="categorical_crossentropy", optimizer=opt,
                  metrics=["accuracy"])
    
    t = now()
    model.fit(x_train, y_train, batch_size=batch_size,
              epochs=epochs, verbose=1,
              validation_data=(x_test, y_test))
    print(f"Training time: {now()-t}")
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test score: {score[0]}")
    print(f"Test accuracy: {score[1]}")

In [6]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [7]:
# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process


feature_layers  =[
    tf.keras.layers.Conv2D(filters, kernel_size, padding="valid",
                           input_shape=input_shape),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(filters, kernel_size),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(pool_size=pool_size),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten()
]

In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes),
    tf.keras.layers.Activation("softmax")
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = tf.keras.models.Sequential(feature_layers + classification_layers)

In [11]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9
train_model(model,
           (x_train_gte5, y_train_gte5),
           (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
train samples: 29404
test samples: 4861
Epoch 1/10
230/230 [==============================] - 13s 56ms/step - loss: 1.6032 - accuracy: 0.2199 - val_loss: 1.5830 - val_accuracy: 0.2253
Epoch 2/10
230/230 [==============================] - 13s 56ms/step - loss: 1.5743 - accuracy: 0.2821 - val_loss: 1.5526 - val_accuracy: 0.3228
Epoch 3/10
230/230 [==============================] - 13s 55ms/step - loss: 1.5459 - accuracy: 0.3611 - val_loss: 1.5202 - val_accuracy: 0.5246
Epoch 4/10
230/230 [==============================] - 13s 56ms/step - loss: 1.5156 - accuracy: 0.4328 - val_loss: 1.4843 - val_accuracy: 0.6468
Epoch 5/10
230/230 [==============================] - 13s 56ms/step - loss: 1.4799 - accuracy: 0.5046 - val_loss: 1.4436 - val_accuracy: 0.7120
Epoch 6/10
230/230 [==============================] - 13s 56ms/step - loss: 1.4417 - accuracy: 0.5522 - val_loss: 1.3976 - val_accuracy: 0.7472
Epoch 7/10
230/230 [==============================] - 13s 55ms

### Freezing Layers
- Keras allows layers to be "frozen" during the training process.
- That is, some layers would have their weights updated during the training process, while others would not.
- This is a core part of transfer learning, the ability to train just the last one or several layers.
<br>
- Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  
- Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.
- This is in addition to the savings gained by being able to train on a smaller data set.

In [13]:
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of **total params**, **trainable params**, and **non-trainable params**.

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [15]:
train_model(model,
           (x_train_lt5, y_train_lt5),
           (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
train samples: 30596
test samples: 5139
Epoch 1/10
240/240 [==============================] - 5s 18ms/step - loss: 1.5550 - accuracy: 0.3390 - val_loss: 1.4929 - val_accuracy: 0.4581
Epoch 2/10
240/240 [==============================] - 4s 18ms/step - loss: 1.4802 - accuracy: 0.4048 - val_loss: 1.4152 - val_accuracy: 0.5353
Epoch 3/10
240/240 [==============================] - 4s 18ms/step - loss: 1.4109 - accuracy: 0.4699 - val_loss: 1.3428 - val_accuracy: 0.6231
Epoch 4/10
240/240 [==============================] - 4s 18ms/step - loss: 1.3478 - accuracy: 0.5346 - val_loss: 1.2753 - val_accuracy: 0.7017
Epoch 5/10
240/240 [==============================] - 4s 17ms/step - loss: 1.2855 - accuracy: 0.5951 - val_loss: 1.2107 - val_accuracy: 0.7605
Epoch 6/10
240/240 [==============================] - 4s 17ms/step - loss: 1.2279 - accuracy: 0.6469 - val_loss: 1.1496 - val_accuracy: 0.8054
Epoch 7/10
240/240 [==============================] - 4s 17ms/step -

- Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.
- This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

- Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, **the training time per epoch was still much reduced.**  
- **This is because the unfrozen part of the network was very shallow, making backpropagation faster.**

## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.

In [16]:
# Create layers and define the model as above
feature_layers2 = [
    tf.keras.layers.Conv2D(filters, kernel_size,
                           padding="valid",
                           input_shape=input_shape),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(filters, kernel_size),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(pool_size=pool_size),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten()
]

classification_layers2 = [
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes),
    tf.keras.layers.Activation("softmax")
]

model2 = tf.keras.models.Sequential(feature_layers2 + classification_layers2)
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [17]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
train samples: 30596
test samples: 5139
Epoch 1/10
240/240 [==============================] - 14s 54ms/step - loss: 1.6094 - accuracy: 0.2051 - val_loss: 1.5838 - val_accuracy: 0.2716
Epoch 2/10
240/240 [==============================] - 13s 54ms/step - loss: 1.5752 - accuracy: 0.2845 - val_loss: 1.5470 - val_accuracy: 0.4585
Epoch 3/10
240/240 [==============================] - 13s 56ms/step - loss: 1.5396 - accuracy: 0.3656 - val_loss: 1.5067 - val_accuracy: 0.5882
Epoch 4/10
240/240 [==============================] - 14s 57ms/step - loss: 1.4993 - accuracy: 0.4549 - val_loss: 1.4607 - val_accuracy: 0.7143
Epoch 5/10
240/240 [==============================] - 13s 56ms/step - loss: 1.4540 - accuracy: 0.5366 - val_loss: 1.4068 - val_accuracy: 0.8060
Epoch 6/10
240/240 [==============================] - 13s 56ms/step - loss: 1.3998 - accuracy: 0.6067 - val_loss: 1.3429 - val_accuracy: 0.8476
Epoch 7/10
240/240 [==============================] - 13s 55ms

In [18]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [19]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [20]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
train samples: 29404
test samples: 4861
Epoch 1/10
230/230 [==============================] - 4s 17ms/step - loss: 1.6053 - accuracy: 0.3025 - val_loss: 1.5367 - val_accuracy: 0.3750
Epoch 2/10
230/230 [==============================] - 4s 17ms/step - loss: 1.5351 - accuracy: 0.3380 - val_loss: 1.4706 - val_accuracy: 0.4232
Epoch 3/10
230/230 [==============================] - 4s 17ms/step - loss: 1.4748 - accuracy: 0.3821 - val_loss: 1.4092 - val_accuracy: 0.4871
Epoch 4/10
230/230 [==============================] - 4s 17ms/step - loss: 1.4185 - accuracy: 0.4347 - val_loss: 1.3510 - val_accuracy: 0.5688
Epoch 5/10
230/230 [==============================] - 4s 17ms/step - loss: 1.3682 - accuracy: 0.4856 - val_loss: 1.2960 - val_accuracy: 0.6336
Epoch 6/10
230/230 [==============================] - 4s 17ms/step - loss: 1.3176 - accuracy: 0.5362 - val_loss: 1.2440 - val_accuracy: 0.6892
Epoch 7/10
230/230 [==============================] - 4s 17ms/step -

***